In [1]:
import os
import json
import pandas as pd
import sys

def load_json_files(directory):
    data_list = []
    file_map = {}

    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".json"):
                file_path = os.path.join(root, filename)
                try:
                    with open(file_path, "r") as f:
                        data = json.load(f)
                        data_list.append(data)
                        file_map[len(data_list) - 1] = file_path  # Track index to file
                except Exception as e:
                    print(f"Error reading {file_path}: {e}", file=sys.stderr)

    if not data_list:
        print("No valid JSON files found.", file=sys.stderr)
        return None

    df = pd.DataFrame(data_list)
    
    nan_rows = df.isna().any(axis=1)
    if nan_rows.any():
        print("Files with NaN values (dropped):")
        for idx in df[nan_rows].index:
            print(f" - {file_map.get(idx, 'Unknown file')}")
        df = df.dropna()  # Drop rows with NaN values

    sorted_df = df.sort_values(by=["m", "k", "n", "world_size"])
    return sorted_df


In [2]:
before_df = load_json_files("../slurm_logs")
after_df = load_json_files("../slurm_logs")

Files with NaN values (dropped):
 - ../slurm_logs/all_reduce_16384-4096-8192_1/all_reduce_16384-4096-8192_1_01302025_234736.json
 - ../slurm_logs/all_reduce_16384-4096-8192_2/all_reduce_16384-4096-8192_2_01302025_234736.json
 - ../slurm_logs/all_reduce_16384-4096-8192_4/all_reduce_16384-4096-8192_4_01302025_234736.json
 - ../slurm_logs/all_reduce_16384-4096-8192_8/all_reduce_16384-4096-8192_8_01302025_234736.json
Files with NaN values (dropped):
 - ../slurm_logs/all_reduce_16384-4096-8192_1/all_reduce_16384-4096-8192_1_01302025_234736.json
 - ../slurm_logs/all_reduce_16384-4096-8192_2/all_reduce_16384-4096-8192_2_01302025_234736.json
 - ../slurm_logs/all_reduce_16384-4096-8192_4/all_reduce_16384-4096-8192_4_01302025_234736.json
 - ../slurm_logs/all_reduce_16384-4096-8192_8/all_reduce_16384-4096-8192_8_01302025_234736.json


In [3]:
columns = ["m", "n", "k", "world_size", "streamk_sms"]
metric_column = "triton_tflops"

merged_df = before_df.merge(after_df, on=columns, suffixes=("_before", "_after"))
merged_df["speedup"] = merged_df[f"{metric_column}_after"] / merged_df[f"{metric_column}_before"]
merged_df = merged_df[columns + ["speedup", "success_before", "success_after"]]


In [4]:
merged_df

,m,n,k,world_size,streamk_sms,speedup,success_before,success_after
0,1024.0,4096.0,512.0,1.0,64.0,1.0,True,True
1,1024.0,4096.0,512.0,2.0,64.0,1.0,True,True
2,1024.0,4096.0,512.0,4.0,64.0,1.0,True,True
3,1024.0,4096.0,512.0,8.0,64.0,1.0,True,True
4,2048.0,8192.0,1024.0,1.0,64.0,1.0,True,True
5,2048.0,8192.0,1024.0,2.0,64.0,1.0,True,True
6,2048.0,8192.0,1024.0,4.0,64.0,1.0,True,True
7,2048.0,8192.0,1024.0,8.0,64.0,1.0,True,True
8,2560.0,10240.0,5120.0,1.0,64.0,1.0,True,True
9,2560.0,10240.0,5120.0,2.0,64.0,1.0,True,True


In [5]:
print(merged_df.to_markdown(index=False))

|    m |     n |    k |   world_size |   streamk_sms |   speedup | success_before   | success_after   |
|-----:|------:|-----:|-------------:|--------------:|----------:|:-----------------|:----------------|
| 1024 |  4096 |  512 |            1 |            64 |         1 | True             | True            |
| 1024 |  4096 |  512 |            2 |            64 |         1 | True             | True            |
| 1024 |  4096 |  512 |            4 |            64 |         1 | True             | True            |
| 1024 |  4096 |  512 |            8 |            64 |         1 | True             | True            |
| 2048 |  8192 | 1024 |            1 |            64 |         1 | True             | True            |
| 2048 |  8192 | 1024 |            2 |            64 |         1 | True             | True            |
| 2048 |  8192 | 1024 |            4 |            64 |         1 | True             | True            |
| 2048 |  8192 | 1024 |            8 |            64 |         1